In [ ]:
import base64
import datetime
import io
import dash
from dash.dependencies import Input, Output, State
from dash import dcc, html, dash_table
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import json
import pandas as pd
#computes the sum for graph   
#----------------------------------------------------------------------------------
#main app
app = dash.Dash(__name__)
app.layout =html.Div([html.Button("Add", id="add", n_clicks=0),'or go to a new page',
                      html.A('Go',href="http://127.0.0.1:8050",target="_blank"),html.Button("View Graph",id="view-graph",n_clicks=0),
                      dcc.Input(id="input",type="text",placeholder="Enter the display you want to see ",value="",debounce="True"),
                      html.Br(),
                      dcc.Input(id="input-csv",type="text",placeholder="Enter the sourcefile name ",value="",debounce="True"),
                      html.Button("View",id="viewcsv",n_clicks=0),
                      html.Div(id="div1",children=[]),
                      html.Div(id="plot3",children=[])])
#--------------------------------------------------------------------------------------------
#adds the main content to upload and gives the buttons
@app.callback(
    Output('div1','children'),
    [
        Input('add', 'n_clicks')
    ], 
    State('div1','children')
)
def display_dropdowns(n_clicks,children):
    if(n_clicks>0):
        new_dropdown = html.Div(id=str(n_clicks),children=[
                            dcc.Upload(
                                id='upload-data',
                                children=html.Div([
                                    'Drag and Drop or ',
                                    html.A('Select Files')
                                ]),
                                style={
                                    'width': '100%',
                                    'height': '60px',
                                    'lineHeight': '60px',
                                    'borderWidth': '1px',
                                    'borderStyle': 'dashed',
                                    'borderRadius': '5px',
                                    'textAlign': 'center',
                                    'margin': '10px'
                                },
                                # Allow multiple files to be uploaded
                                multiple=True
                            ),
                            html.Button("Choose Graph",id="add-graph",n_clicks=0),
                            dcc.Input(id="input1",type="text",placeholder="Enter the display name",value="",debounce="True"),
                            html.Button("Choose Columns",id="chose-column",n_clicks=0),
                            html.Button("View Contents",id="content",n_clicks=0),
                            html.Div(id='output-data-upload2',children=[]),
                            html.Div(id='output-data-upload4',children=[]),
                            #html.Div(id='output-data-upload4',children=[html.H5("CONTENTS:")]),
                            html.Div(id='output-data-upload3',children=[]),
                            html.Div(id='output-data-upload',children=[]),
                            html.Div(children=[ ], id='plot1'),
                            html.Div(children=[ ], id='plot2'),
                                
                            ])
        children.append(new_dropdown)
    return children
#------------------------------------------------------------------------------------------------------------------------
#display the dropdowns on clicking buttons
@app.callback(
    Output('output-data-upload','children'),
    [
        Input('add-graph','n_clicks'),
        Input('upload-data','filename')
        #Input("input-csv",'value')
    ],
    State('output-data-upload','children'))

def graphing(n_clicks,filename,children):
    if(n_clicks>0):
        axis_list = list(pd.read_csv(filename[0], nrows =1))
    if(n_clicks>0):
        new=html.Div(id='drop',children=[html.H1('Dashboard', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 40}),
                                html.Br(),
                                html.Div([
                                        html.Div(
                                            [
                                            html.H2('x axis:', style={'margin-right': '2em'}),
                                            ]
                                        ),
                                        dcc.Dropdown(id='x-axis', 
                                                    options=[
                                                    {'label': i, 'value': i} for i in axis_list],
                                                    placeholder='Select x axis',
                                                    style={'width':'80%','height':'3px', 'font-size': '20px', 'text-align-last' : 'center'})
                                                     ], style={'display':'flex'}),
                                html.Div([
                                        html.Div(
                                            [
                                            html.H2('y axis:', style={'margin-right': '2em'}),
                                            ]
                                        ),
                                        dcc.Dropdown(id='y-axis', 
                                                    options=[
                                                     {'label': i, 'value': i} for i in axis_list],
                                                    placeholder='Select y axis',
                                                    style={'width':'80%','height':'3px', 'font-size': '20px', 'text-align-last' : 'center'})
                                                     ], style={'display':'flex'}),
                                html.Div([
                                        html.Div(
                                            [
                                            html.H2('Chart Type:', style={'margin-right': '2em'}),
                                            ]
                                        ),
                                        dcc.Dropdown(id='chart-type', 
                                                    options=[
                                                    {'label': 'Bar Graph', 'value': 'OPT1'},
                                                    {'label': 'Pi Chart', 'value': 'OPT2'},
                                                    {'label': 'Scatter Plot', 'value': 'OPT3'},
                                                    
                                                            ],
                                                    placeholder='Select a Chart type',
                                                    style={'width':'80%','height':'3px', 'font-size': '20px', 'text-align-last' : 'center'})
                                                     ], style={'display':'flex'}),#html.Div(children=[ ], id='plot3'),
                                html.Div([
                                        html.Div(
                                            [
                                            html.H2('Hierarchy:', style={'margin-right': '2em'}),
                                            ]
                                        ),
                                        dcc.Dropdown(id='hier', 
                                                    options=[
                                                     {'label': i, 'value': i} for i in axis_list],
                                                    placeholder='Select Hierarchy',
                                                    style={'width':'80%','height':'3px', 'font-size': '20px', 'text-align-last' : 'center'})
                                                     ], style={'display':'flex'}),
                                html.Button("Save my choice ",id='save',n_clicks=0),
                                html.Button("Save my Hierarchy ",id='save1',n_clicks=0)
                                ])
        children.append(new)
    return children
#-----------------------------------------------------------------------------------------------------------
#function when save my choice is pressed to store it 
@app.callback(
              Output('save','n_clicks'),
              [
                  Input('save','n_clicks'),
                  Input('input1','value'),
                  Input(component_id='chart-type', component_property='value'),
                  Input(component_id='x-axis', component_property='value'),
                  Input(component_id='y-axis', component_property='value'),
              ])
def storing(n_clicks,value0,value1,value2,value3):
    if(n_clicks>0):
        fn=str(value0)+".json"
        f=open(fn,"a")
        dict1={'value':value0,'x-axis':value2,'y-axis':value3,'chart':value1}
        f.write(json.dumps(dict1))
        f.write("\n")
        f.close()
    return 0
@app.callback(
              Output('save1','n_clicks'),
              [
                  Input('save1','n_clicks'),
                  Input('input1','value'),
                  Input('hier','value')
              ])
def storingHier(n_clicks,value0,value1):
    if(n_clicks>0):
        fn=str(value0)+".json"
        f=open(fn,"a")
        dict1={'hier':value1}
        f.write(json.dumps(dict1))
        f.write("\n")
        f.close()
    return 0

#-----------------------------------------------------------------------------------------------------
#Displyas the graph based on dropdown options
@app.callback([
                Output(component_id='plot1', component_property='children'),
                Output(component_id='plot2', component_property='children'),
              ],
              [
                #Input(component_id='chart-type', component_property='value'),
                #Input(component_id='x-axis', component_property='value'),
                #Input(component_id='y-axis', component_property='value'),
                #Input('upload-data','filename'),
                Input('view-graph','n_clicks'),
                Input('input','value'),
                #Input('upload-data','filename'),
                Input('input-csv','value')
              ],
              [
                State("plot1", "children"), 
                State("plot2", "children")
              ])
def get_graph(n_clicks,value1,filename,children1,children2):
    #print(filename[0])
    try:
        if(n_clicks>0):
            fn=str(value1)+".json"
            f=open(fn,"r")
            fp=f.readlines()
            list_dict=[]
            for j in range(0,len(fp)):
                dictionary1=json.loads(fp[j])
                list_dict.append(dictionary1)
            for index1 in range(0,len(list_dict)):
                if(list(list_dict[index1].keys())[0]!='column'and list(list_dict[index1].keys())[0]!='hier'):
                    fp=list_dict[index1] 
            df =  pd.read_csv(filename,encoding = "ISO-8859-1")
            x = df[fp['x-axis']]
            y = df[fp['y-axis']]
            if fp['chart'] == 'OPT1':
                graph = px.bar(df, x=x, y=y, color=y, title='Bar Chart')
            elif fp['chart'] == 'OPT2':
                graph = px.pie(df, values=y, names=x, title='Pie Chart')
            elif fp['chart'] == 'OPT3':
                graph = px.scatter(df, x=x, y=y, color=y, title='Scatter Chart')
            f.close()
            fn=str(value1)+".json"
            f=open(fn,"r")
            fp=f.readlines()
            list_path=[]
            list_send=[]
            no_hier_col=0
            for m in range(0,len(fp)):
                dictionary1=json.loads(fp[m])
                list_path.append(dictionary1)
            print(list_path)
            for index2 in range(0,len(list_path)):
                if(list(list_path[index2].keys())[0]=='hier'):
                    #fp=list_path[index2]
                    list_send.append(list_path[index2]['hier'])
            print(list_send)
            #list_hiers=[]
            #for n in range(0,no_hier_col):
                #list_hiers.append(df[fp['hier']])
            #a = df[fp['hie1']]
            #b = df[fp['hie2']]
            #c = df[fp['hie3']]
            #bar_data, scat_data, pi_data, tree_data = compute_data_choice_1(df)
            #print(list_dict[i]['chart'])
            tree_fig = px.treemap(df, path=list_send, 
                                 values=y,
                                 color=y,
                                 color_continuous_scale='RdBu',
                                 title='Tree Graph')
            return[dcc.Graph(figure=graph),dcc.Graph(figure=tree_fig)]
    except:
        return(html.P("There was an error displaying the graph as the given file name and the stored file columns don't match"),html.P(""))

#-------------------------------------------------------------------------------------------------------------------
#displays the contents of the file
def parse_contents(contents, filename, date):
    content_type,content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),

        dash_table.DataTable(
            df.to_dict('records'),
            [{'name': i, 'id': i} for i in df.columns]
        ),

        html.Hr(),  # horizontal line

        # For debugging, display the raw contents provided by the web browser
        html.Div('Raw Content'),
        html.Pre(contents[0:200] + '...', style={
            'whiteSpace': 'pre-wrap',
            'wordBreak': 'break-all'
        })
    ])
#-------------------------------------------------------------------------------------------------------------------
#displyas the csv file in a row 
"""@app.callback(Output('output-data-upload4', 'children'),
            [Input('viewcsv','n_clicks'),
            Input('upload-data','filename'),
            Input('col','value')],
            State('output-data-upload4', 'children'))
def display_select(n_clicks,filename,value,children):
    if(n_clicks):
        df=pd.read_csv(filename[0])
       # print(type(value)) it's str
        new=df[value]
        new3=str(new).split(",")
        new2=html.Div(id=str(n_clicks),children=new3)
        children.append(new2)
    return children"""
#----------------------------------------------------------------------------------------------------------------------
#Gives the dropdown for csv columns
@app.callback(Output('output-data-upload3', 'children'),
            [Input('chose-column','n_clicks'),
            #Input('input-csv','value')
            Input('upload-data','filename')
            ],
            State('output-data-upload3', 'children'))
def select_few(n_clicks,filename,children):
    if(n_clicks>0):
        axis_list = list(pd.read_csv(filename[0], nrows =1))
    if(n_clicks>0):
        new=html.Div(id='drop',children=[html.H1('CSV Display', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 40}),
                                html.Br(),
                                html.Div([
                                        html.Div(
                                            [
                                            html.H2('CSV column:', style={'margin-right': '2em'}),
                                            ]
                                        ),
                                        dcc.Dropdown(id='col', 
                                                    options=[
                                                    {'label': i, 'value': i} for i in axis_list],
                                                    placeholder='Select column',
                                                    style={'width':'80%','height':'3px', 'font-size': '20px', 'text-align-last' : 'center'})
                                                     ], style={'display':'flex'}),
                                        html.Div(id='output-data-upload4',children=[]),
                                        html.Button("Save",id="savecsv",n_clicks=0)])
                                        
        children.append(new)
    return children
#------------------------------------------------------------------------------------------------------------------------
#To save into the file the CSV columns
@app.callback(Output('savecsv','n_clicks'),
              Input('col','value'),
              Input('input1','value'),
              Input('savecsv','n_clicks'),
              )
def savecsv_col(value1,value2,n_clicks2):
    if(n_clicks2>0):
        fn=str(value2)+".json"
        f=open(fn,"a")
        dict1={'column':value1}
        f.write(json.dumps(dict1))
        f.write("\n")
        f.close()
    return 0

#------------------------------------------------------------------------------------------------------------------------
#display only that CSV
@app.callback(Output('output-data-upload4', 'children'),
              Input('viewcsv','n_clicks'),
              Input('input','value'),
              #State('upload-data', 'filename'),
              State('input-csv','value')
              )
def update_output_few(n_clicks,filename,list_of_names):
    try:
        if(n_clicks):
            fn=str(filename)+".json"
            f=open(fn,"r")
            fp=f.readlines()
            #fp=json.loads(fp[2,])
            #print(type(children))
            list_dict=[]
            list_col=[]
            found=0
            i=0
            for j in range(0,len(fp)):
                dictionary1=json.loads(fp[j])
                list_dict.append(dictionary1)
                #print(list_dict)
                #print(dictionary1['value'],list_dict[i]['value'])
                #print(type(dictionary1['value']))
            for index1 in range(0,len(list_dict)):
                if(list(list_dict[index1].keys())[0]=='column'):
                    list_col.append(list_dict[index1]['column'])
            #print(list_keys)
            #if(list_keys[0]=='column'):
                    #print(dictionary1['value'],value1)
                #list_col.append(dictionary1[list_keys[i]])
            #print(list_col)
            df=pd.read_csv(list_of_names,usecols=list_col)
            #new=df[value]
            #new2=str(new).split(',')
            #for i in new2:
                #print(i)
            #print(new)
            #children.append(html.Div(id='new22',children=new))
            return html.Div([dash_table.DataTable(
                df.to_dict('records'),
                [{'name': i, 'id': i} for i in df.columns]
            ),

            html.Hr()])
        """if new is not None:
            #print(list_of_contents) it's a list 
            children = [
                parse_contents(c, n, d) for c, n, d in
                zip(new, list_of_names, list_of_dates)]
    return children"""
   
        #df=pd.read_csv(list_of_names[0])
       # print(type(value)) it's str
        #new=df[value]
        #new3=str(new).split(",")
        #new2=html.Div(id=str(n_clicks),children=new3)
        #children.append(new2)
    except:
        return(html.P("There was an error as the given file and the columns stored don't match"))
#------------------------------------------------------------------------------------------------------------------------- 
# displays the full csv  
@app.callback(Output('output-data-upload2', 'children'),
              Input('upload-data', 'contents'),
              Input('content','n_clicks'),
              #State('upload-data', 'filename'),
              State('input-csv','value'),
              State('upload-data', 'last_modified'),
              State('output-data-upload2', 'children'))
def update_output(list_of_contents,n_clicks,list_of_names, list_of_dates,children):
    if(n_clicks):
        if list_of_contents is not None:
            #print(list_of_contents) it's a list 
            children = [
                parse_contents(c, n, d) for c, n, d in
                zip(list_of_contents, list_of_names, list_of_dates)]
    return children
#--------------------------------------------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server('0.0.0.0', debug = False)

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.198:8050/ (Press CTRL+C to quit)
192.168.0.198 - - [27/Aug/2022 00:43:29] "GET / HTTP/1.1" 200 -
192.168.0.198 - - [27/Aug/2022 00:43:29] "GET /_dash-layout HTTP/1.1" 200 -
192.168.0.198 - - [27/Aug/2022 00:43:29] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.0.198 - - [27/Aug/2022 00:43:29] "GET /_favicon.ico?v=2.5.0 HTTP/1.1" 200 -
192.168.0.198 - - [27/Aug/2022 00:43:29] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.198 - - [27/Aug/2022 00:44:34] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.198 - - [27/Aug/2022 00:44:35] "GET /_dash-component-suites/dash/dcc/async-upload.js HTTP/1.1" 200 -
192.168.0.198 - - [27/Aug/2022 00:44:35] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.198 - - [27/Aug/2022 00:44:35] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.198 - - [27/Aug/2022 00:44:35] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.198 - - [27/Aug/2022 00:44:35] "POST /_dash-u

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\snigd\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\snigd\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\snigd\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\snigd\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\snigd\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\snigd\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\snigd\anaconda3\lib\site-packages\dash\dash.py", line 1468, in dispatch
    response.set_d

192.168.0.198 - - [27/Aug/2022 00:44:35] "POST /_dash-update-component HTTP/1.1" 500 -
192.168.0.198 - - [27/Aug/2022 00:44:42] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.198 - - [27/Aug/2022 00:44:42] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.198 - - [27/Aug/2022 00:44:42] "POST /_dash-update-component HTTP/1.1" 200 -
